# DNN Critic model

In [2]:
import pandas as pd 
import numpy as np  
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# read data
mydt = pd.read_csv("../dataset/UGR/traindos.csv")
print(mydt['Label'].value_counts())

# Create matrix of features and matrix of target variable 
dataset = mydt.values
X = dataset[:,0:19]
y = dataset[:,19]

# convert the scaled array to dataframe 
#min_max_scaler = MinMaxScaler()
#X_scale = min_max_scaler.fit_transform(X)

X_full = X
y_full = y

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.3, random_state=1)

print(Counter(y_train))
print(Counter(y_test))

# define the keras model
model = Sequential()
model.add(Dense(128, input_shape=(19,), activation='relu'))
model.add(Dense(19, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Building Critic WGAN model...")
# fit the keras model on the dataset
mod = model.fit(X_train, y_train, epochs=100, batch_size=30, verbose=1, validation_data=(X_test, y_test))

# make class predictions with the model
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100))

%matplotlib inline
# Testing 
cm = confusion_matrix(y_test, y_pred) 
tnr = recall_score(y_test, y_pred) 
fpr = 1 - tnr

print ("confusion_matrix for test data\n",cm)
print(classification_report(y_test,y_pred,labels=np.unique(y_pred)))
print('Accuracy: %.4f' % accuracy_score(y_test, y_pred))
print('Precision: %.4f' % precision_score(y_test, y_pred))
print('Recall: %.4f' % recall_score(y_test, y_pred))
print('F1 Score: %.4f' % f1_score(y_test, y_pred))
print('FPR Score: %.4f' % fpr)

# Save model
# serialize model to JSON
model_json = model.to_json()
with open("DNN_critic_model_UGR_1.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("DNN_critic_model_UGR_1.h5")
print("Saved model!")

0    14029
1     3165
Name: Label, dtype: int64
Counter({0.0: 14029, 1.0: 3165})
Counter({0.0: 5971, 1.0: 570})
Building Critic WGAN model...
Epoch 1/100
574/574 [==============================] - 4s 5ms/step - loss: 0.0556 - accuracy: 0.9814 - val_loss: 0.8411 - val_accuracy: 0.9145
Epoch 2/100
574/574 [==============================] - 3s 5ms/step - loss: 0.0074 - accuracy: 0.9974 - val_loss: 1.2082 - val_accuracy: 0.9113
Epoch 3/100
574/574 [==============================] - 3s 5ms/step - loss: 0.0058 - accuracy: 0.9981 - val_loss: 1.4055 - val_accuracy: 0.9116
Epoch 4/100
574/574 [==============================] - 3s 5ms/step - loss: 0.0055 - accuracy: 0.9977 - val_loss: 1.4971 - val_accuracy: 0.9115
Epoch 5/100
574/574 [==============================] - 3s 5ms/step - loss: 0.0045 - accuracy: 0.9983 - val_loss: 1.8163 - val_accuracy: 0.9118
Epoch 6/100
574/574 [==============================] - 3s 5ms/step - loss: 0.0046 - accuracy: 0.9984 - val_loss: 1.8371 - val_accuracy: 0.9112


Epoch 57/100
574/574 [==============================] - 3s 5ms/step - loss: 0.0016 - accuracy: 0.9993 - val_loss: 4.0453 - val_accuracy: 0.9122
Epoch 58/100
574/574 [==============================] - 2s 4ms/step - loss: 0.0012 - accuracy: 0.9997 - val_loss: 4.2462 - val_accuracy: 0.9119
Epoch 59/100
574/574 [==============================] - 3s 5ms/step - loss: 0.0014 - accuracy: 0.9995 - val_loss: 4.0883 - val_accuracy: 0.9122
Epoch 60/100
574/574 [==============================] - 3s 6ms/step - loss: 0.0016 - accuracy: 0.9994 - val_loss: 3.8660 - val_accuracy: 0.9113
Epoch 61/100
574/574 [==============================] - 3s 5ms/step - loss: 0.0011 - accuracy: 0.9997 - val_loss: 4.1693 - val_accuracy: 0.9122
Epoch 62/100
574/574 [==============================] - 3s 4ms/step - loss: 0.0016 - accuracy: 0.9992 - val_loss: 4.1959 - val_accuracy: 0.9118
Epoch 63/100
574/574 [==============================] - 3s 5ms/step - loss: 0.0010 - accuracy: 0.9997 - val_loss: 4.2724 - val_accuracy:

# WGAN

In [2]:
import torch 
import torch.nn
import torch.nn.functional as nn 
import torch.autograd as autograd 
import torch.optim as optim 
import torch.utils.data as Data
from torch.autograd import Variable
import numpy as np 
import tensorflow as tf 
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import confusion_matrix 
import keras
from keras.models import Sequential 
from keras.layers import Dense 
from keras import optimizers
from keras.models import model_from_json
import matplotlib 
matplotlib.use('agg') 
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec 

# hyperparameters 
mb_size = 64 
z_dim = 10 
X_dim = 19 
y_dim = 1 
h_dim = 128 #hidden 
lr = 0.00005 #learning_rate

# read data
df = pd.read_csv("../dataset/UGR/traindos.csv")
print('original data label features number:')
print(df['Label'].value_counts())

# data preparation
criteria0 = df['Label'] == 1 #filter attack data 
attack_data = data0 = df[criteria0]

#float_array = data0.values.astype(float) 
#scaler = MinMaxScaler()
#scaled_array = scaler.fit_transform(float_array)
#attack_data = pd.DataFrame(scaled_array)
#attack_data.columns = ['hour','minute','second','duration','source_ip','dest_ip','source_port','dest_port','protocol',
#                'flag1','flag2','flag3','flag4','flag5','flag6','fwd','stos','pkt','byt', 'Label']

print('----------------------------------------') 
print('attack value number:') 
print(attack_data['Label'].value_counts())

# transform data to tensor format
train_target = torch.tensor(attack_data['Label'].values.astype(np.float32)) 
train = torch.tensor(attack_data.drop(['Label'], axis = 1).values.astype(np.float32)) 
train_tensor = Data.TensorDataset(train, train_target) 
train_tensor = Data.TensorDataset(train, train_target) 

#creat DataLoader with batch and shuffle
train_loader = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = mb_size, shuffle = True)

# define models
# Generator
G = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim), 
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, h_dim), 
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, X_dim), 
    torch.nn.Sigmoid()
)

# Discriminator
D = torch.nn.Sequential( 
    torch.nn.Linear(X_dim, h_dim), 
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, h_dim), 
    torch.nn.ReLU(),
    # No sigmoid
    torch.nn.Linear(h_dim, 1),
)

# setup cpu using cpu (if use gpu -> set cpu to cuda)
G.cpu()
D.cpu()

def reset_grad():
    G.zero_grad()
    D.zero_grad()
    
#initialize optimizers and dataholders
G_solver = optim.RMSprop(G.parameters(), lr=lr)
D_solver = optim.RMSprop(D.parameters(), lr=lr)
G_losses = []
D_losses = []
data_iter = iter(train_loader)

original data label features number:
0    14029
1     3165
Name: Label, dtype: int64
----------------------------------------
attack value number:
1    3165
Name: Label, dtype: int64


In [3]:
#training 
print("Start training WGAN model...")
d_iter = 2 
for it in range(500000):
    if it<100: d_iter = 50 
    else:
        d_iter = 2
    for _ in range(d_iter):
        # Sample data
        z = Variable(torch.randn(mb_size, z_dim)).cpu()
        try:
            X, _ = data_iter.next() 
        except StopIteration:
            data_iter = iter(train_loader) 
            X, _ = data_iter.next()
        #X = Variable(torch.from_numpy(X))
        X = X.cpu()
        
        # Dicriminator forward-loss-backward-update 
        G_sample = G(z)
        D_real = D(X)
        D_fake = D(G_sample)
        
        D_loss = -(torch.mean(D_real) - torch.mean(D_fake))
        
        D_loss.backward() 
        D_solver.step()
        
        # Weight clipping 
        for p in D.parameters():
            p.data.clamp_(-0.01, 0.01)
            
        D_losses.append(D_loss.data.cpu().numpy())
        
        # Housekeeping - reset gradient 
        reset_grad()
        
    # Generator forward-loss-backward-update 
    #X = Variable(torch.from_numpy(X)) 
    z = Variable(torch.randn(mb_size, z_dim)).cpu() #change cpu to cuda
    
    G_sample = G(z)
    D_fake = D(G_sample)
    
    G_loss = -torch.mean(D_fake)
    
    G_loss.backward() 
    G_solver.step()

    # Housekeeping - reset gradient 
    reset_grad()

    G_losses.append(G_loss.data.cpu().numpy())

    # Print and plot every now and then 
    if it % 1000 == 0:
        print('Iter-{}; D_loss: {}; G_loss: {}'
              .format(it, D_loss.data.cpu().numpy(),  
                G_loss.data.cpu().numpy()))
        
        samples = G(z).data.numpy()[:16]

#plot losses
plt.figure(figsize=(10,5))
plt.title("Generator and Critic Loss During Training") 
plt.plot(G_losses,label="Generator") 
plt.plot(D_losses,label="Critic") 
plt.xlabel("iter")
plt.ylabel("loss")
plt.legend()
plt.savefig('UGR_loss_1.png')

#save model
torch.save({'modelG_state_dict': G.state_dict(), 
            'modelD_state_dict': D.state_dict(),
            'optimizerG_state_dict': G_solver.state_dict(),
            'optimizerD_state_dict': D_solver.state_dict(),
            'G_loss': G_losses,
            'D_loss': D_losses
           }, "WGAN_model_UGR_1.pth")

Start training WGAN model...
Iter-0; D_loss: -0.0015891212970018387; G_loss: -0.009423905983567238
Iter-1000; D_loss: -0.00016767904162406921; G_loss: -0.005721299909055233
Iter-2000; D_loss: -0.0020155105739831924; G_loss: -0.008379052393138409
Iter-3000; D_loss: -0.0005434677004814148; G_loss: -0.009727489203214645
Iter-4000; D_loss: -0.00034055300056934357; G_loss: -0.010439911857247353
Iter-5000; D_loss: -0.0002306560054421425; G_loss: -0.010126546025276184
Iter-6000; D_loss: -8.835271000862122e-05; G_loss: -0.01046947855502367
Iter-7000; D_loss: -0.00025398097932338715; G_loss: -0.010341127403080463
Iter-8000; D_loss: 6.786361336708069e-05; G_loss: -0.010601264424622059
Iter-9000; D_loss: -0.00015464425086975098; G_loss: -0.010325468145310879
Iter-10000; D_loss: 1.2930482625961304e-05; G_loss: -0.010469404980540276
Iter-11000; D_loss: -4.951655864715576e-05; G_loss: -0.010514343157410622
Iter-12000; D_loss: -3.626011312007904e-05; G_loss: -0.010486874729394913
Iter-13000; D_loss: 

Iter-111000; D_loss: -2.494174987077713e-05; G_loss: -0.013234879821538925
Iter-112000; D_loss: 3.201887011528015e-06; G_loss: -0.01332034170627594
Iter-113000; D_loss: 1.9672326743602753e-05; G_loss: -0.013147389516234398
Iter-114000; D_loss: -1.5581026673316956e-06; G_loss: -0.013194399885833263
Iter-115000; D_loss: 7.071811705827713e-05; G_loss: -0.01297986600548029
Iter-116000; D_loss: -3.11313197016716e-05; G_loss: -0.013067707419395447
Iter-117000; D_loss: 8.380971848964691e-06; G_loss: -0.01313625369220972
Iter-118000; D_loss: 1.6848556697368622e-05; G_loss: -0.012965427711606026
Iter-119000; D_loss: -1.5121884644031525e-05; G_loss: -0.013059526681900024
Iter-120000; D_loss: -1.3339333236217499e-05; G_loss: -0.013058625161647797
Iter-121000; D_loss: -1.0225921869277954e-06; G_loss: -0.012971863150596619
Iter-122000; D_loss: -3.048870712518692e-05; G_loss: -0.012962643057107925
Iter-123000; D_loss: 5.706213414669037e-06; G_loss: -0.012839016504585743
Iter-124000; D_loss: 8.567236

Iter-221000; D_loss: 3.3704563975334167e-06; G_loss: -0.012944425456225872
Iter-222000; D_loss: -1.4957040548324585e-06; G_loss: -0.012861699797213078
Iter-223000; D_loss: -7.92182981967926e-06; G_loss: -0.0127492630854249
Iter-224000; D_loss: -3.0656345188617706e-05; G_loss: -0.013036116026341915
Iter-225000; D_loss: -4.1564926505088806e-05; G_loss: -0.012908601202070713
Iter-226000; D_loss: -2.4260953068733215e-06; G_loss: -0.012669308111071587
Iter-227000; D_loss: -1.0266900062561035e-05; G_loss: -0.012832575477659702
Iter-228000; D_loss: 1.2850388884544373e-05; G_loss: -0.012729673646390438
Iter-229000; D_loss: 2.048257738351822e-05; G_loss: -0.012164120562374592
Iter-230000; D_loss: -1.0191462934017181e-05; G_loss: -0.01253239531069994
Iter-231000; D_loss: 1.8617138266563416e-06; G_loss: -0.012325849384069443
Iter-232000; D_loss: 2.4255365133285522e-05; G_loss: -0.012627669610083103
Iter-233000; D_loss: 1.8440186977386475e-06; G_loss: -0.012536419555544853
Iter-234000; D_loss: -1.

Iter-331000; D_loss: 3.4244731068611145e-06; G_loss: -0.01250424049794674
Iter-332000; D_loss: 1.4804303646087646e-05; G_loss: -0.012566830031573772
Iter-333000; D_loss: -3.419816493988037e-06; G_loss: -0.012513961642980576
Iter-334000; D_loss: -2.1578744053840637e-06; G_loss: -0.012658575549721718
Iter-335000; D_loss: 5.653128027915955e-07; G_loss: -0.012594136409461498
Iter-336000; D_loss: -1.1993572115898132e-05; G_loss: -0.012385421432554722
Iter-337000; D_loss: -2.4978071451187134e-06; G_loss: -0.012444835156202316
Iter-338000; D_loss: 3.475695848464966e-06; G_loss: -0.012520967051386833
Iter-339000; D_loss: 3.641471266746521e-07; G_loss: -0.012416154146194458
Iter-340000; D_loss: -2.5993213057518005e-06; G_loss: -0.012414462864398956
Iter-341000; D_loss: -2.255663275718689e-06; G_loss: -0.012538244016468525
Iter-342000; D_loss: 3.3574178814888e-06; G_loss: -0.01249597780406475
Iter-343000; D_loss: -1.2498348951339722e-06; G_loss: -0.012508071959018707
Iter-344000; D_loss: 1.12596

Iter-441000; D_loss: 5.616806447505951e-06; G_loss: -0.01292978785932064
Iter-442000; D_loss: 1.1962838470935822e-05; G_loss: -0.012848182581365108
Iter-443000; D_loss: 3.07522714138031e-06; G_loss: -0.012823309749364853
Iter-444000; D_loss: -3.0528753995895386e-06; G_loss: -0.012967310845851898
Iter-445000; D_loss: 4.5318156480789185e-06; G_loss: -0.012822826392948627
Iter-446000; D_loss: 4.092231392860413e-06; G_loss: -0.012815754860639572
Iter-447000; D_loss: 9.462237358093262e-07; G_loss: -0.012775127775967121
Iter-448000; D_loss: 1.8421560525894165e-06; G_loss: -0.01274191029369831
Iter-449000; D_loss: 4.51505184173584e-06; G_loss: -0.012855791486799717
Iter-450000; D_loss: 1.1101365089416504e-06; G_loss: -0.012700976803898811
Iter-451000; D_loss: 2.039596438407898e-07; G_loss: -0.012884944677352905
Iter-452000; D_loss: -5.140900611877441e-07; G_loss: -0.012912482023239136
Iter-453000; D_loss: 3.287568688392639e-07; G_loss: -0.012799344956874847
Iter-454000; D_loss: -4.51412051916

# Data generation from saved WGAN model

In [4]:
#loaded NN model
print("Loading model...")
print("----------------------------------------")
json_file = open("DNN_critic_model_UGR_1.json","r") 
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json) 
loaded_model.load_weights("DNN_critic_model_UGR_1.h5")

G = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, X_dim), 
    torch.nn.Sigmoid()
)

checkpoint = torch.load("WGAN_model_UGR_1.pth")
G.load_state_dict(checkpoint['modelG_state_dict'])

G.eval()
G.cpu()

final_Data = pd.DataFrame(columns = ['hour','minute','second','duration','source_ip',
                                     'dest_ip','source_port','dest_port',
                                     'protocol','flag1','flag2','flag3','flag4','flag5',
                                     'flag6','fwd','stos','pkt','byt','Label']) 
print("empty df",final_Data) 
print("----------------------------------------")
print("Completed loading model")

Loading model...
----------------------------------------
empty df Empty DataFrame
Columns: [hour, minute, second, duration, source_ip, dest_ip, source_port, dest_port, protocol, flag1, flag2, flag3, flag4, flag5, flag6, fwd, stos, pkt, byt, Label]
Index: []
----------------------------------------
Completed loading model


In [5]:
it =0

while(final_Data.shape[0]<17000):
    it+=1
    z = Variable(torch.randn(100, z_dim)).cpu() 
    with torch.no_grad():
        data = G(z)
    data = data.cpu().numpy() 
    fake = pd.DataFrame(data)
        
    newy = np.ones((fake.shape[0],)) 
    #this is the model which changes 
    new_pred = loaded_model.predict(fake)
    new_pred = (new_pred > 0.5)
    cm = confusion_matrix(newy,new_pred)
    print("-------------------------------------------------------------") 
    print (" confusion_matrix for new data with deeper model \n" ,cm)
    
    combined_data = np.concatenate((fake,new_pred),axis=1) 
    combined_data = pd.DataFrame(combined_data)
    combined_data.columns = ['hour','minute','second','duration','source_ip',
                             'dest_ip','source_port','dest_port','protocol',
                             'flag1','flag2','flag3','flag4','flag5','flag6',
                             'fwd','stos','pkt','byt', 'Label']
    
    criteria1 = combined_data['Label'] == 1 #just attack data 
    data1 = combined_data[criteria1]
    print("only attack data - ",data1.shape) 
    final_Data = final_Data.append(data1, ignore_index = True) 
    print("final data ", it, "- ",final_Data.shape) 
    print("-------------------------------------------------------------") 

print(final_Data['Label'].value_counts())
print("---------------------------------------------------------")
df2 = final_Data.sample(n=10864, replace=False, random_state=1)
df3 = final_Data.sample(n=17000, replace=False, random_state=1)
final = pd.concat([mydt,df2], ignore_index = True,axis=0)
final['Label'].value_counts()
df3.to_csv("../dataset/UGR/generated_dos.csv", index = False, header = True)
final.to_csv("../dataset/UGR/balanced_data_dos.csv", index = False, header = True)

4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[100]]
only attack data -  (100, 20)
final data  1 -  (100, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [ 1 99]]
only attack data -  (99, 20)
final data  2 -  (199, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[100]]
only attack data -  (100, 20)
final data  3 -  (299, 20)
-------------------------------------------------------------


/tmp/ipykernel_1583204/645084960.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_1583204/645084960.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_1583204/645084960.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)


4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[100]]
only attack data -  (100, 20)
final data  4 -  (399, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [ 2 98]]
only attack data -  (98, 20)
final data  5 -  (497, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[100]]
only attack data -  (100, 20)
final data  6 -  (597, 20)
-------------------------------------------------------------


/tmp/ipykernel_1583204/645084960.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_1583204/645084960.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_1583204/645084960.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)


4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[100]]
only attack data -  (100, 20)
final data  7 -  (697, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[100]]
only attack data -  (100, 20)
final data  8 -  (797, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 2ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[100]]
only attack data -  (100, 20)
final data  9 -  (897, 20)
-------------------------------------------------------------


...

/tmp/ipykernel_1583204/645084960.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_1583204/645084960.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)


4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[100]]
only attack data -  (100, 20)
final data  168 -  (16773, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[100]]
only attack data -  (100, 20)
final data  169 -  (16873, 20)
-------------------------------------------------------------


/tmp/ipykernel_1583204/645084960.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_1583204/645084960.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)


4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[ 0  0]
 [ 1 99]]
only attack data -  (99, 20)
final data  170 -  (16972, 20)
-------------------------------------------------------------
4/4 [==============================] - 0s 3ms/step
-------------------------------------------------------------
 confusion_matrix for new data with deeper model 
 [[100]]
only attack data -  (100, 20)
final data  171 -  (17072, 20)
-------------------------------------------------------------
1.0    17072
Name: Label, dtype: int64
---------------------------------------------------------


/tmp/ipykernel_1583204/645084960.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
/tmp/ipykernel_1583204/645084960.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_Data = final_Data.append(data1, ignore_index = True)
